In [1]:
import torch
print(torch.cuda.device_count())
from unsloth import FastLanguageModel

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig
import transformers
from datasets import load_dataset, Dataset
import pprint
import json
from pathlib import Path
import os
import numpy as np
import time
import re
import pandas as pd

2
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:

device = f"cuda"
from huggingface_hub import hf_hub_download
import os


if torch.cuda.get_device_capability()[0] >= 8:
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
# attn_implementation = "eager"
print(f"Using {attn_implementation} for attention computation.")

# ------------------------------------------------------------

model_id = 'sh2orc/Llama-3.1-Korean-8B-Instruct'
# model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
# model_id = 'Saxo/Linkbricks-Horizon-AI-Korean-Gemma-2-sft-dpo-27B'
# model_id = 'Bllossom/llama-3-Korean-Bllossom-70B'

model_dir = f"/model/{model_id.replace('/', '-')}"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    # max_seq_length = max_seq_length,
    dtype = torch_dtype,
    load_in_4bit = False,
    load_in_8bit = False,
    # quantization_config=BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_use_double_quant=True,
    #     bnb_4bit_quant_type="nf4",
    #     bnb_4bit_compute_dtype=torch_dtype
    #     # load_in_8bit=True,
    #     # llm_int8_enable_fp32_cpu_offload=False if not "27B" in model_id else True,
    # ),
    # device_map=device,
    cache_dir=f"{model_dir}/cache",
    attn_implementation=attn_implementation,
    local_files_only=True
)
FastLanguageModel.for_inference(model)
tokenizer.padding_side = "left"

Using flash_attention_2 for attention computation.
==((====))==  Unsloth 2025.2.5: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.689 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

sh2orc/Llama-3.1-Korean-8B-Instruct does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [3]:
from pathlib import Path  # Import Path from pathlib
import json
dataset_name = "v7-250309-reduceinputanddatefunctioncall"
dataset_dir = Path(f"/workspace/finetuning/dataset/{dataset_name}")

metadatas = {}
for directory in dataset_dir.iterdir():
    if directory.is_dir() and "scenario" in directory.name:
        with open(f"{directory}/metadata.json", "r", encoding="utf-8") as f:
            metadatas[directory.name] = json.loads(f.read())

print(metadatas)
        

{'scenario1': {'site_name': 'YongDongIllHighSchool', 'user_name': '홍길동', 'user_role': 'customer', 'idu_name': '01_IB5', 'idu_mapping': {'01_IB5': ['우리반'], '01_IB7': ['옆반'], '02_I81': ['앞반']}, 'modality_mapping': {'roomtemp': ['실내온도'], 'settemp': ['설정온도'], 'oper': ['전원']}, 'current_datetime': '2022-09-30 12:00:00'}}


In [4]:
file_name = "db_gt_v7.json"
file_path = f"/workspace/experiments/{file_name}"

with open(file_path, "r") as f:
    datas = json.load(f)


In [ ]:
import time
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
system_prompt = """

너는 질문에 대해 주어진 결과값을 참고해 정직하고 자연스러운 답변을 제공하는 인공지능 모델이다.
숫자는 소숫점 2자리까지 표시해.
만약 결과의 특정 변수가 공란/None이거나 특정 변수의 element 개수가 10개보다 많다면, 해당 변수는 건너뛰어도 무방해.
온도는 음수가 나올수있어. 무시하지 말고 정직하게 답해.
이 외의 경우에는 결과를 왜곡하지 말고 정직하게 답해.
공손하게 대답해야해.
예시를 참고해 최대한 모든 변수를 다루도록 노력해.
예시를 참고하지만 꼭 같을 필욘 없어.
반말로 물어볼지라도, 존댓말로 답변해줘.
또한 답변에 여러개의 timestamp 가 있는 경우에 (10개 이상) 대표값 몇개만 출력해줘. 
예시 : 
질문: 2주전 우리반과 옆반 합쳐서 설정온도가 가장 낮은날이 언제야?
Metadata: {'site_name': 'YongDongIllHighSchool', 'user_name': '홍길동', 'user_role': 'customer', 'idu_name': '01_IB5', 'idu_mapping': {'01_IB5': ['우리반'], '01_IB7': ['옆반'], '02_I81': ['앞반']}, 'modality_mapping': {'roomtemp': ['실내온도'], 'settemp': ['설정온도'], 'oper': ['전원']}, 'current_datetime': '2022-09-30 12:00:00'}
예시: ['2주전 우리반과 옆반 합쳐서 설정온도가 가장 낮은 날은 {{coldest_dates}}로 {{coldest_temp}}℃였습니다.']
결과: {'coldest_dates': ['2022-09-14', '2022-09-13', '2022-09-12', '2022-09-18', '2022-09-17', '2022-09-16', '2022-09-15'], 'coldest_temp': 23.0}
답변:
2주전 우리반과 옆반 합쳐서 설정온도가 가장 낮은 날은 2022-09-14, 2022-09-13, 2022-09-12, 2022-09-18, 2022-09-17, 2022-09-16, 2022-09-15 로, 23.0℃였습니다.

"라고 답변해야 합니다." ,"답변은 다음과 같습니다" 이런말 빼줘. 어떤 지시사항을 말해주는게 아니라 사용자에게 바로 답변을 해줘야해.
없는 정보에 대해서는 추측을 하는 답변 조차 하지 말고, HVAC 시스템과 관련 없는 질문(농담 등)에 대해서는 HVAC과는 관련없는 정보라고 해줘. 
추론 과정은 나타낼 필요 없어. 최종 답만 나타내.
결과가 빈 리스트면, 답변을 만들지말고, 해당 값이 없다고 답변해줘.
실제 답변을 할때에는, 예시를 그대로 따라하지말고, 변수에 해당하는 수로 바꾸어서 정확하게 답변해야해. 변수명을 쓰는게 아니라 변수에 담긴 값을 확인하고 답변을 해줘. 꼭 중요해. 예시에 담긴 변수명을 그대로 쓴 답변은 하지마.변수명을 그대로 출력하는 것이 아니라, 실제 값을 기반으로 답변을 만들어야 해.중요해. 변수명 답변에 그대로 쓰지마 절대.
답변을 할때, 질문을 한번더 쓸 필요는 없어.
결과가 빈 리스트라면 (예를 들어, 결과: {'dates': []}), 해당 데이터가 없습니다. 라고 표현해야해. 이상한 수를 만들거나 예시를 그대로 따라하지마.
사용자가 언급한 개념(예: "뒷반")이 주어진 메타데이터(`idu_mapping`, `modality_mapping` 등)에서 정확히 어떤 항목을 의미하는지 확인하고,만약 사용자의 질문에서 필요한 정보(예: 특정 장소, 날짜 범위 등)가 명확하지 않다면, 먼저 해당 정보를 요청해줘. 예를 들어, 사용자가 "뒷반 온도가 가장 높은 날이 언제야?"라고 물었을 때 "뒷반"이 어느 반인지 특정할 수 없으면, `"뒷반이 어디인지 알려주세요."`와 같이 추가 정보를 요청해줘. 
만일 날짜상으로, 미래에 대한 정보를 요청한다면 정보를 제공할 수 없다고 해줘. 추론 하지말고 없는 위치, 미래 날짜 등에 대해서는 정보가 없다고 해줘.

만일 왜이렇게 더워? 왜이렇게 추워? 등의 질문을 받으면 주어진 실내온도(roomtemp), 설정온도(settemp), 에어컨 작동 여부(oper), 경과 시간(elapsed_minutes, elapsed_seconds), 그리고 이전 데이터를 기반으로, 한 문장으로 답변을 작성하세요. 예시로:

avg_room_temp = result['roomtemp'].mean()
avg_set_temp = result['settemp'].mean()
current_room_temp = result['roomtemp'].iloc[-1]
current_set_temp = result['settemp'].iloc[-1]

왜이렇게 더워? (실내 온도가 높음) 
1. 에어컨이 꺼져 있음 (result['oper']==false)
"주어진 시간 동안 평균 우리반의 온도는 계속 average_room_temp℃였고, 에어컨이 꺼져 있어서 실내 온도가 내려가지 않았어요. 에어컨을 켜면 시원해질 거에요."

2. 설정 온도가 너무 높음
"주어진 시간 평균 우리반 온도는 average_room_temp℃였는데, 평균 에어컨 설정 온도가 avg_set_temp℃로 높아 냉방 효과가 크지 않았어요. 설정 온도를 낮추면 시원해질 거에요."

3. 에어컨이 켜져 있고 설정 온도가 실내 온도보다 낮지만 점점 시원해지는 중
"주어진 시간 우리반 온도는 average_room_temp℃였고, 현재 에어컨이 켜져 있으며 설정 온도가 avg_set_temp℃로 낮아요. 에어컨이 가동된 지 얼마 안 돼서 아직 실내 온도가 내려가는 중일 수 있어요. 조금만 기다리면 더 시원해질 거에요."

왜 이렇게 추워? (실내 온도가 낮음) 
1. 에어컨이 꺼져 있음 (result['oper']==false)
"주어진 시간 평균 우리반 온도는 average_room_temp℃였고, 에어컨이 꺼져 있지만 실내 온도가 계속 낮았어요. 창문을 닫거나 난방을 가동하면 따뜻해질 거에요."

2. 설정 온도가 너무 낮음
"주어진 시간 평균 우리반 온도는 average_room_temp℃였는데, 평균 에어컨 설정 온도가 avg_set_temp℃로 낮아 실내가 더욱 추웠어요. 설정 온도를 높이면 따뜻해질 거에요."

3. 에어컨이 켜져 있고 설정 온도가 실내 온도보다 높지만 점점 따뜻해지는 중
"주어진 시간 우리반 온도는 average_room_temp℃였고, 현재 에어컨이 켜져 있으며 설정 온도가 avg_set_temp℃로 높아요. 에어컨이 가동된 지 얼마 안 돼서 아직 실내 온도가 올라가는 중일 수 있어요. 조금만 기다리면 더 따뜻해질 거에요."

"""
input_template = """
질문: {input}
Metadata: {metadata}
예시: {expectations}
결과: {result}
"""
# 결과에는 음수등 의도치 않은 결과가 포함되어 있을 수 있으니, 결과를 왜곡하지 말고 정직하게 답해주세요.

for data in datas:
    start_time = time.time()
    input, tags, scenario, result = data["Input"], data["Tags"], data["Scenario"], data["Result"]
    
    tags = [tags['Type']] if isinstance(tags, dict) else tags
    pass_tags = ["Normal", "Unknown", "Metadata", "NoData", "Unrelated"]
    pass_ = False
    for pass_tag in pass_tags:
        if pass_tag in tags:
            pass_ = True
            break
    if not pass_:
        continue
    
    print("====================================================")
    metadata = metadatas[scenario]

    variables = {}
    for instruction in result:
        i_type = instruction["type"]
        if i_type == "o":
            for k, v in instruction["returns"].items():
                type_, value = v
                # print(k, type_, value)
                if type_ == "pd":
                    # restore pd.DataFrame.to_dict(orient="index") or pd.Series.to_dict()
                    value = pd.DataFrame.from_dict(value, orient="index")
                elif type_ in ["primitive"]:
                    pass
                else:
                    raise NotImplementedError(f"Unsupported type: {k, type_, value}")
                variables[k] = value
            variables.update()

            # print(input, variables)
        elif i_type == "r":
            expectations = instruction["expectations"]
            user_input = input_template.format(
                input=input,
                expectations=expectations,
                result=variables,
                metadata=metadata
            )

            chat = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ]

            print(user_input)

            inputs = tokenizer.apply_chat_template(
                    chat,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to(model.device)

            outputs = model.generate(
                input_ids=inputs,
                max_new_tokens=3000,
                use_cache=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                do_sample=False  # 결정론적 생성
            )
            
            # 결과 디코딩 및 파싱
            responses = tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]

            pattern = r"<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>"
            match = re.search(pattern, responses, re.DOTALL)
            result = match.group(1).strip() if match else None

            end_time = time.time()
            print("답변:")
            print(result)
            print("Time elapsed:", end_time - start_time)
            




질문: 어제 우리반과 옆반의 설정온도 차이 알려줘
Metadata: {'site_name': 'YongDongIllHighSchool', 'user_name': '홍길동', 'user_role': 'customer', 'idu_name': '01_IB5', 'idu_mapping': {'01_IB5': ['우리반'], '01_IB7': ['옆반'], '02_I81': ['앞반']}, 'modality_mapping': {'roomtemp': ['실내온도'], 'settemp': ['설정온도'], 'oper': ['전원']}, 'current_datetime': '2022-09-30 12:00:00'}
예시: ['어제 우리반의 설정온도({{settemp_ours}}℃)는 옆반의 설정온도({{settemp_beside}}℃)보다 {{settemp_diff}}℃ 높습니다.']
결과: {'settemp_ours': 23.0, 'settemp_beside': 23.0, 'settemp_diff': 0.0}

답변:
어제 우리반의 설정온도(23.0℃)는 옆반의 설정온도(23.0℃)보다 0.0℃ 높습니다.
Time elapsed: 1.4660255908966064

질문: 오늘 우리반과 옆반의 평균 온도차이 알려줘
Metadata: {'site_name': 'YongDongIllHighSchool', 'user_name': '홍길동', 'user_role': 'customer', 'idu_name': '01_IB5', 'idu_mapping': {'01_IB5': ['우리반'], '01_IB7': ['옆반'], '02_I81': ['앞반']}, 'modality_mapping': {'roomtemp': ['실내온도'], 'settemp': ['설정온도'], 'oper': ['전원']}, 'current_datetime': '2022-09-30 12:00:00'}
예시: ['우리반({{roomtemp_ours}}℃)이 옆반({{roomtemp_beside}}℃)보다 {{r

KeyboardInterrupt: 